In [1]:
import pandas as pd
attacks = pd.read_csv(r'attacks.csv', encoding='latin-1')

In [9]:
attacks.isna()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
1,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,True
2,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,True,True
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
25719,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
25720,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
25721,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
